In [1]:
from pathlib import Path
import json
import glob
import numpy as np
import librosa
from librosa import display
import matplotlib.pyplot as plt
from DLNet_functions import preprocess_wrapper
import tensorflow as tf
assert tf.__version__ >= "2.0"
# autotune computation
AUTOTUNE = tf.data.experimental.AUTOTUNE



In [2]:
# Create Config for preprocessing and pipeline parameters

config = {'sr': 44100,
          'audio_length': 1,
          'mono': True,
          'n_mels': 64,
          'n_fft': 1024,
          'hop_length': 256,
          'win_length': 512,
          'window': 'hann',
          'center': True,
          'pad_mode': 'reflect',
          'power': 2.0,
          }

# save classes from foldernames
# folders = glob.glob('_data/*_wav/')
# config['classes'] = sorted(set([Path(f).parts[-1] for f in folders]))
config['classes'] = ['compressed_wav', 'uncompr_wav']

# save number of frames from length in samples divided by fft hop length
config['n_frames'] = int(
    config['sr']*config['audio_length']/config['hop_length']) + 1

# save input shape for model
config['input_shape'] = (config['n_mels'], config['n_frames'], 1)

# save config
with open('DLNet_config.json', 'w+') as fp:
    json.dump(config, fp, sort_keys=True, indent=4)

# Creater wrapper object:
ds_config = 'dl4aed_project/Code/_data/dataset_config.json'
wrapper = preprocess_wrapper(config, ds_config)

In [3]:
# Generate datasets (true) or load datasets (false):
generate = True
if generate:
    # Generate mp3_32k dataset and uncompressed wav dataset from dirs
    # this dataset will not be saved!
    dirs = ['_data/compressed_wav/mp3_32k',
            '_data/uncompr_wav']
    dataset = wrapper.gen_tf_dataset_from_list(dirs)
else:
    # Load dataset:
    dataset_1 = wrapper.load_tf_dataset('_data/dataset_mp3_32k')
    dataset_2 = wrapper.load_tf_dataset('_data/dataset_uncompr_wav')
    dataset = dataset_2.concatenate(dataset_1)


InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: _data/compressed_wav/mp3_32k/*.wav'

In [ ]:
# VISUALIZE WAVEFORMS
# get all wav files
fps = glob.glob('_data/*_wav/**/*.wav', recursive=True)
fps_random = []
np.random.seed(9)

# setup subplot
nrows, ncols = 2, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 6))

# plot some audio waveforms
for r in range(nrows):
    for c in range(ncols):
        fp_random = fps[np.random.randint(len(fps))]
        audio, sr = librosa.core.load(fp_random, sr=None)
        ax[r][c].plot(audio, c='k')
        # ax[r][c].axis('off')
        ax[r][c].set_title(Path(fp_random).parts[-2:])
        if r == 0:
            ax[r][c].set_xticks([])
        # save random audio filepaths
        fps_random.append(fp_random)


In [5]:
# VISUALIZE SPECTROGRAMS
# setup subplot
nrows, ncols = 4, 2
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 12))

# plot some audio waveforms
for i, fp_random in enumerate(fps_random):
    audio, sr = librosa.core.load(fp_random, sr=None)

    # calculate stft
    stft = librosa.stft(audio, n_fft=config['n_fft'],
                        hop_length=config['hop_length'],
                        win_length=config['win_length'])

    # calculate melspec
    melspec = librosa.feature.melspectrogram(audio, n_fft=config['n_fft'],
                                             hop_length=config['hop_length'],
                                             n_mels=config['n_mels'],
                                             fmax=int(config['sr']/2))
    melspec = librosa.amplitude_to_db(melspec, ref=np.max)

    # calculate magnitude and scale to dB
    magspec = librosa.amplitude_to_db(np.abs(stft), ref=np.max)

    # plot with librosa
    librosa.display.specshow(magspec, x_axis='time', y_axis='linear', sr=sr,
                             hop_length=256, ax=ax[i][0])
    librosa.display.specshow(melspec, x_axis='time', y_axis='mel', sr=sr,
                             hop_length=256, ax=ax[i][1])

    # adjustments
    # ax[i][1].set_yticks([])
    ax[i][1].set_ylabel(Path(fp_random).parts[-2], rotation=270, labelpad=20)
    ax[i][1].yaxis.set_label_position("right")

    # settings for all axises but bottom ones
    if not i == len(fps_random) - 1:
        ax[i][0].set_xticks([])
        ax[i][1].set_xticks([])
        ax[i][0].set_xlabel('')
        ax[i][1].set_xlabel('')

    # settings for upper axises
    if i == 0:
        ax[i][0].set_title('stft')
        ax[i][1].set_title('mel spectrogram')

# adjust whitespace in between subplots
plt.subplots_adjust(hspace=0.1, wspace=0.1)

print('Melspec shape: %s' % (str(melspec.shape)))
print('Stft shape: %s' % (str(stft.shape)))
print(f'Total data points in mel-spectrogram: \
      {melspec.shape[0]*melspec.shape[1]}')
print(f'Total data points in stft-spectrogram: {stft.shape[0]*stft.shape[1]}')
print(f'-> Data Reduction by factor: \
      {(stft.shape[0]*stft.shape[1]) / (melspec.shape[0]*melspec.shape[1])}')



NameError: name 'melspec' is not defined

In [ ]:
# Split dataset in 80:20 (test:train)
buff_size = len(dataset)
train_size = int(.8*buff_size)
test_size = buff_size - train_size
# shuffle before splitting in train and eval dataset
dataset = dataset.shuffle(buffer_size=buff_size)
dataset = dataset.cache()

# take first 80% from dataset
train_dataset = dataset.take(train_size)
train_dataset = train_dataset.shuffle(buffer_size=train_size)
train_dataset = train_dataset.batch(64)
train_dataset = train_dataset.prefetch(AUTOTUNE)

# take last 20% samples from dataset
test_dataset = dataset.skip(test_size).shuffle(test_size)
test_dataset = test_dataset.batch(64).prefetch(AUTOTUNE)
eval_dataset = test_dataset


In [7]:
# create model architecture
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=config['input_shape']))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.GaussianDropout(0.25))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.GaussianDropout(0.25))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu"))
model.add(tf.keras.layers.GlobalMaxPool2D())
model.add(tf.keras.layers.Dense(len(config['classes']), activation="sigmoid"))

# Define metrics
metrics = [tf.keras.metrics.TrueNegatives(),
           tf.keras.metrics.TruePositives(),
           tf.keras.metrics.FalseNegatives(),
           tf.keras.metrics.FalsePositives(),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
           tf.keras.metrics.CategoricalAccuracy()
           ]

# compile model
n_epochs = 10
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# fit model
history = model.fit(train_dataset, epochs=n_epochs,
                    validation_data=eval_dataset)


NameError: name 'train_dataset' is not defined

In [ ]:
# setup plot
fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(16,4))

# plot loss
ax[0].plot(range(n_epochs), history.history['loss'])
ax[0].plot(range(n_epochs), history.history['val_loss'])
ax[0].set_ylabel('loss'), ax[0].set_title('train_loss vs val_loss')

# plot accuracy
ax[1].plot(range(n_epochs), history.history['categorical_accuracy'])
ax[1].plot(range(n_epochs), history.history['val_categorical_accuracy'])
ax[1].set_ylabel('accuracy'), ax[1].set_title('train_acc vs val_acc')

# plot adjustement
for a in ax:
    a.grid(True)
    a.legend(['train','val'], loc=4)
    a.set_xlabel('num of Epochs')
plt.show()
